In [1]:
import numpy as np
import torch

# visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import corner

%load_ext autoreload
%autoreload 2

In [2]:
# remove top and right axis from plots
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False

In [3]:
import pyro
from pyro.distributions import Beta, Binomial, HalfCauchy, Normal, Pareto, Uniform
from pyro.distributions.util import scalar_like
from pyro.infer import MCMC, NUTS, Predictive
from pyro.infer.mcmc.util import initialize_model, summary
from pyro.util import ignore_experimental_warning

In [4]:
import lbi.models
from lbi.nde import NeuralRatioEstimator, NeuralLikelihoodEstimator
from lbi.sequential import Sequential

In [5]:
def simulator(parameter_set, **kwargs):
    obs_dim = 8
    
    m_theta = parameter_set[:, :2]
    s1 = parameter_set[:, 2].pow(2)    
    s2 = parameter_set[:, 3].pow(2)
    rho = torch.tanh(parameter_set[:, 4])
    S_theta = [torch.stack([s1.pow(2), rho*s1*s2], dim=-1), 
               torch.stack([rho*s1*s2, s2.pow(2)], dim=-1)]
    S_theta = torch.stack(S_theta, dim=-1)

    # TODO: Speed this up
    samples = []
    for m_th, S_th in zip(m_theta, S_theta):
        dist = torch.distributions.MultivariateNormal(m_th, S_th)
        sample = dist.sample((obs_dim//2,)).flatten()
        samples.append(sample)

    return torch.stack(samples)

In [6]:
param_dim = 5
obs_dim = 8
true_param = torch.tensor([[0.7, -2.9, -1., -0.9, 0.6]])
observation = simulator(true_param)

In [7]:
# Make multivariate uniform prior distribution
priors = torch.distributions.Independent(torch.distributions.Uniform(low=-3*torch.ones(param_dim), high=3*torch.ones(param_dim)), 
                                         reinterpreted_batch_ndims=1)

In [8]:
layers = [['Linear', obs_dim + param_dim, 64], ['SELU'], 
          ['Linear', 64, 64], ['SELU'], 
          ['Linear', 64, 1]]
model = lbi.models.Classifier(layers=layers)
optimizer = torch.optim.AdamW(model.parameters())
nre = NeuralRatioEstimator(model)

In [9]:
snre = Sequential(priors=priors, obs_data=observation, param_dim=param_dim, model=nre, optimizer=optimizer, 
                  simulator=simulator, 
                  n_rounds=10,
                  sims_per_model=1,
                  num_initial_samples=500,
                  num_samples_per_round=1000)

In [ ]:
snre.run()

Training on 425 samples. Validating on 75 samples.


Warmup:   7%|▋         | 21/300 [00:00, 209.57it/s, step size=3.23e-01, acc. prob=0.734]

Round 1 complete. Time elapsed: 0m 1s. Total time elapsed: 0m 1s.


Sample: 100%|██████████| 300/300 [00:02, 128.67it/s, step size=7.24e-02, acc. prob=0.850]


Training on 1,275 samples. Validating on 225 samples.


...

Training on 5,525 samples. Validating on 975 samples.


Warmup:   2%|▏         | 5/300 [00:00, 25.22it/s, step size=3.12e-03, acc. prob=0.398]

Round 7 complete. Time elapsed: 0m 15s. Total time elapsed: 1m 2s.


Sample: 100%|██████████| 300/300 [00:02, 144.84it/s, step size=2.21e-01, acc. prob=0.756]


Training on 6,375 samples. Validating on 1,125 samples.


In [ ]:
posterior_samples = snre.hmc(num_samples=200000, walker_steps=7500, burn_in=500)

In [ ]:
corner.corner(posterior_samples.numpy(), 
              range=[(-3, 3) for i in range(param_dim)], 
              truths=true_param[0].numpy());

In [ ]:
model = lbi.models.ConditionalFlow(dim=obs_dim, 
                                   context_dim=param_dim, 
                                   transform_type='autoregressive', 
                                   n_layers=1, 
                                   hidden_units=128,
                                   n_blocks=2, 
                                   dropout=0., 
                                   use_batch_norm=False, 
                                   tails='linear', 
                                   tail_bound=5, 
                                   n_bins=10,
                                   min_bin_height=1e-5, 
                                   min_bin_width=1e-5, 
                                   min_derivative=1e-5, 
                                   unconditional_transform=True,
                                   encoder=None)
optimizer = torch.optim.Adam(model.parameters())
nle = NeuralLikelihoodEstimator(model)

In [ ]:
snle = Sequential(priors=priors, obs_data=observation, param_dim=param_dim, model=nle, optimizer=optimizer, 
                  simulator=simulator, 
                  reset_every_round=True,
                  n_rounds=10,
                  sims_per_model=1,
                  max_n_epochs=200,
                  num_initial_samples=500,
                  num_samples_per_round=1000)

In [ ]:
snle.run()

In [ ]:
posterior_samples = snle.hmc(num_samples=200000, walker_steps=2500, burn_in=500)

In [ ]:
corner.corner(posterior_samples.numpy(), 
              range=[(-3, 3) for i in range(param_dim)], 
              truths=true_param[0].numpy());